In [11]:
import os, sys, argparse, paramiko
from getpass import getpass
from os.path import exists

In [12]:
# Create the password prompt
thePass = getpass(prompt="Please enter your SSH password:  ")

# Host information
host = "192.168.6.71"
port = 2222
username = "sarah.fornaldes"
password = thePass

try:
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

except paramiko.AuthenticationException:
    print("Authentication failed")

In [13]:
sftp = ssh.open_sftp()

# Run a command to scan all known directories on Linux that have a *bin* directory
command = '/home/ubuntu/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin'

ssh.exec_command('kraken')
sftp.put('kraken', 'kraken')

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

# Get results  from stdout
lines =  stdout.readlines()
#print(lines)

# Convert the list to a string
output =  ''.join(lines)
#print(output)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output =  sepHeader + output +  sepFooter
#print(cmd_output)

# Save the cmd_output to a file
with open('results.txt', 'a') as f:
    f.write(cmd_output)

#ssh.close()
sftp.close()

In [14]:
sftp = ssh.open_sftp()

# Uploads results.txt file
uploadFiles = ['results.txt']

# Runs the lsof command, lsof lists all open files for a process                
command = 'sudo -S lsof -i -n'

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)

# Input password to run the command as sudo, including a new line character that provides "Enter" as if you were typing it in on a keyboard
stdin.write(thePass + "\n")

# Get results  from stdout
lines =  stdout.readlines()

# Convert the list to a string
output =  ''.join(lines)

# Header output
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'

# Footer output
sepFooter = '' + '### END ' + command + ' ###\n\n'

# Concatonate the header, output, and footer
cmd_output =  sepHeader + output +  sepFooter

# Save the cmd_output to a file
with open('resultsLsof.txt', 'a') as f:
    f.write(cmd_output)

# End SFTP Session
#ssh.close()
sftp.close()

In [15]:
sftp = ssh.open_sftp()

# Uploads results.txt file
uploadFiles = ['results.txt']

# Runs the lsof command, lsof lists all open files for a process                
command = 'sudo -S lsof -i -n -p XXXX'

# Get the output from the command
stdin, stdout, stderr  = ssh.exec_command(command)
stdin.write(thePass + "\n")
lines =  stdout.readlines()
output =  ''.join(lines)
sepHeader = '' + '### BEGIN ' + command + ' ###\n\n'
sepFooter = '' + '### END ' + command + ' ###\n\n'
cmd_output =  sepHeader + output +  sepFooter

# Save the cmd_output to a file
with open('resultsLsof-p.txt', 'a') as f:
    f.write(cmd_output)

# End SFTP Session
ssh.close()
sftp.close()

look at threat report, use regex for lsof -p xxxx command
https://github.com/Sarah-F/SYS-320-01/blob/main/Week5/Homework/searchLogCsv.py 

In our cases, the files have been under 600K.  In some cases, executables were discovered with a high entropy (random) filename but have only been seen in lowercase letters between 6 and 17 characters and are packed using UPX. 

https://docs.google.com/document/d/1qFpv0056RU098Vg0x5ASPkOeyFVz5zOzw4qUyEima54/edit

Task 3: If there are any processes running containing matches from the task above for ThugStyle activity, use lsof to list all open files for that process.  - done

Look for detections that include:

pid=XXXX

Where XXX is the process ID or (pid). The command “lsof” must be run as root or a user with sudo permissions to see details of a process owned by another user. The Ubuntu user for this assignment has sudo privileges.  In your video submission, be sure to show the lsof output.

Done most of this, need to isolate pid


Task 4: If any results are returned in the task above that are suspicious (based on known activity), write a program that downloads all suspicious files, including the suspicious binary and any log files associated with the processes.


Task 5: Check running processes, the /etc/passwd file, and /var/log/auth.log file for known activity for ThugStyle.  Respond to the questions below:


Filesystem activity

-ThugStyle behaviors are associated with creating an account with the format x.xxxxx.  The first character has always been a letter, followed by a dot and real last names, sometimes containing a 01 suffix. Authentication to the account uses SSH.  

-The malicious files are pseudo-random where it is a filename of a well known Linux command such as ps, cp, etc. but it is stored in a “*/bin*” directory to attempt to hide its existence.  In our cases, the files have been under 600K.  In some cases, executables were discovered with a high entropy (random) filename but have only been seen in lowercase letters between 6 and 17 characters and are packed using UPX.  Packers like UPX are used to compress executables, though threat actors use it to make identifying attributes of malicious files difficult because the compression hides API calls and other useful data to create signatures on.  Analysis of the file in those cases discovered it was used to exploit a local privilege vulnerability that led to administrative access.


Network activity
A few binaries have been known to listen on a well known port which is a simple web server that performs specific actions like executing a shell or system command, it is used as a staging host for exfiltrating files (transferring files out of the organization) to random IPs.  The listening executable is always owned by the user x.xxxxx.  Commands to the web server  are base64 encoded and usually executes a command shell or downloads and uploads files.  In all cases, where there is the web server, there have been other compromised hosts on the LAN.  All of the binaries are written in Go. If Go is not used extensively within the organization, then scans for Go binaries can be performed.


Discussion

In the above scenario, Yara can be used to search for Go binaries, files packed with UPX, and/or under the 600K filesize.  Yara is a python program that reads in a set of rules and scans files, processes, or memory for patterns matching the rules.  It is a defacto tool used in malware analysis, incident handling, and by security analysts.

The following Yara rules are used to find ThugStyle activity.  Remember that unless you have really specific information to put into a Yara rule, it could generate a lot of false positives.  The more specific you are, the more likely you’ll find anomalies with few false positives.  When results are found, the question to think on is, for example, “Is that the normal location for that file on the specific OS version?”  Are files usually packed?  If so, what is the packer most commonly used?  With Yara, you can filter for specific versions of a packer as small changes are made as the program is updated.  Also, packing files is not that unusual.  Is it unusual for the specific OS and the file found?

The Go application Kraken can be used to scan remote systems for threats because it supports scanning currently running processes and the filesystem.  It also compiles in the YARA rules so a single executable can be uploaded to a system and scanned for threat actor artifacts.

The command “lsof” is a very powerful tool.  The command can be used to enumerate details about a process, list network sockets, etc.  The command stands for “LiSt Open Files.”  If you run:

 lsof -p XXXX


Where XXX is a process ID.  It will list *all* open files, libraries, devices, etc. associated with the process.
